In [1]:
from selenium import webdriver
import pandas as pd
import re

EXE_PATH = "/Users/aprakash/Documents/personal/chromedriver"

In [2]:
def goodreads_login(driver, username, password):
    """
    Login using credentials
    """
    driver.find_element_by_id("user_email").send_keys('arushi.prakash307@gmail.com')
    driver.find_element_by_id("user_password").send_keys("stupid@123")
    driver.find_element_by_name("next").click()
    
def go_to_page(driver, pagename):
    """
    Head to the webpage of choice
    """
    driver.get(pagename)
    
def click_till_disabled(driver, class_name):
    """
    Given that the page is open, finds the first element
    of class name and clicks on it, returns True.
    If element is not found, returns False.
    """
    import selenium.common.exceptions as selexcept 
    try:
        els = driver.find_element_by_class_name(class_name)
        if 'disabled' in els.get_attribute('class'):
            return False
        els.click()
    except selexcept.NoSuchElementException:
        return False
    return True


In [3]:
def open_new_tab(driver, page_name, tab_location):
    """
    Opens a new tab and switches focus to it, given the tab location
    """
    # Opened a new tab
    driver.execute_script('''window.open("about:blank", "_blank");''')
    # Switched focus to the new tab
    driver.switch_to_window(driver.window_handles[tab_location])
    # Opened the user profile
    driver.get(page_name)

In [4]:
def regex_token(text, prefix, suffix):
    """
    Given a user_id link in GoodReads format, extract userid
    """
    extracted_ = re.search(prefix+r'(.*?)'+suffix, text).group(1)
    return extracted_

In [ ]:
def get_book_genre(driver, href):
    """
    Open a book tab and extract the genres, and return csv delimited list
    """
    n_tabs = len(driver.window_handles)
    open_new_tab(driver, href, n_tabs)
    
    els = driver.find_elements_by_class_name('actionLinkLite.bookPageGenreLink')
    genres = []
    for el in els:
        if 'users' not in el.text:
            genres.append(el.text)

    driver.close()
    driver.switch_to_window(driver.window_handles[n_tabs - 1])
    return genres
    
def get_books_on_page(driver):
    """
    Given a books list page, return all the book names
    """
    
    column_names = ['Book Name', 'Book Link']
    data = pd.DataFrame(columns=column_names)
    els = driver.find_elements_by_class_name('field.title')
    for el in els:
        try:
            book_name = el.find_element_by_class_name('value').text
            book_link = el.find_element_by_class_name('value').find_element_by_tag_name('a').get_attribute('href')
            this_data = pd.DataFrame([[book_name, book_link]], columns=column_names)
            data = data.append(this_data, ignore_index=True)
        except:
            pass
    return data

In [ ]:
#Step by Step
# Step 1.  Go to the login page and sign in
driver = webdriver.Chrome(executable_path=EXE_PATH)
go_to_page(driver,
           pagename='https://www.goodreads.com/user/sign_in')
goodreads_login(driver,
                username="",
                password="")
# Step 2. Go to friends page to iterate over all friends
go_to_page(driver,
           pagename='https://www.goodreads.com/friend')
# Step 3. For each friend, extract userid and go to their read list
column_names = ['Book Name', 'Book Link']
all_book_data = pd.DataFrame(columns=column_names)
CLASS_NAME = 'next_page'
next_icon = True
N_FRIENDS = 0
while next_icon:
    # Analyze each friend in this page
    friends = driver.find_elements_by_class_name('userLink')
    for friend in friends:
        N_FRIENDS +=1
        href = friend.get_attribute('href')
        userid = regex_token(href, 'show/', '-')
        new_link = "https://www.goodreads.com/review/list/{}?shelf=read".format(userid)
        # Open new tab for each friend's read list
        n_tabs = len(driver.window_handles)
        open_new_tab(driver, new_link, n_tabs)
        # Get the first twenty books
        books = get_books_on_page(driver)
        all_book_data = all_book_data.append(books, ignore_index=True)
        # Close the tab for friends reading list
        driver.close()
        driver.switch_to_window(driver.window_handles[n_tabs - 1])
        
    # Go to next page if it exists
    next_icon = click_till_disabled(driver, CLASS_NAME)
# Sample page: https://www.goodreads.com/review/list/63180448?shelf=read



/Users/aprakash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  
/Users/aprakash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: DeprecationWarning: use driver.switch_to.window instead


In [ ]:
# Step 4. For the book names collected, get the genres
genres_list = []
for index, row in all_book_data.iterrows():
    #print(row['Book Link'])
    genres = get_book_genre(driver, row['Book Link'])
    all_book_data.loc[index,'Genre'] = ','.join(genres)
    genres_list.extend(genres)

/Users/aprakash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.window instead
  
/Users/aprakash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: use driver.switch_to.window instead
  from ipykernel import kernelapp as app


In [ ]:
# Just write it all to csv to save whatever I have
all_book_data.to_csv('data.csv')

In [ ]:
#driver = webdriver.Chrome(executable_path=EXE_PATH)
#go_to_page(driver,
#           pagename='https://www.goodreads.com/user/sign_in')
#goodreads_login(driver,
#                username="",
#                password="")


In [ ]:
genre_data = pd.DataFrame(genres_list, columns=['Genre'])
genre_data['Genre'].value_counts()

In [ ]:
all_book_data['Book Name'].value_counts()